Exercise 1: Basic Parallel Vector Operations with MPI (5Points)
    
    Suppose you are given a vector v ∈ R N . Initialize your vector v with random numbers (can be either inte-
gers or floating points). You will experiment with three different sizes of vector, i.e. N = {10 7 , 10 12 , 10 15 }.
You have to parallelize vector operations mentioned below using MPI API. For each operation you will
have to run experiment with varying number of workers, i.e. if your system has P workers than run
experiments with workers = {1, 2, . . . P } for each size of vector given above. You have to time your
code for each operation and present it in a table. [Note: You have to define/explain your parallelization
strategy i.e. how you assign task to each worker, how you divide your data etc.]. You have to use MPI
point-to-point communication i.e. Send and Recv.

a) Add two vectors and store results in a third vector.

In [ ]:
%%file Ex1a.py
'''Exercise 1: Basic Parallel Vector Operations with MPI'''
'''a) Add two vectors and store results in a third vector'''

#Import numpy,mpi4py and time packages
import numpy as np
from mpi4py import MPI
import time

#Setting alias to MPI.COMM_WORLD for easier reference in the code
comm = MPI.COMM_WORLD

#Computing the number of processes(size) and rank(current process)
size = comm.Get_size()
rank = comm.Get_rank()

print(comm,size,rank)
resultVec = []

#Function to through the different receiver processes to send the data
def broadcast(data1,data2,size):
    for i in range(1,size):
        #Calculate processing time at each process
        start = MPI.Wtime()
        #Evenly spacing the vector among the available workers
        startIndex = int((i-1)*(len(data1)/(size-1)))
        endIndex = int(((len(data1)/(size-1))*i))
        ##print("Index",startIndex,endIndex)
        #May cause index out of range due to index strting from 0, but we are counting from 1
        if endIndex > len(data1):
            endIndex = int(len(data1))
        ##print("Data Sent :",data[startIndex:endIndex])
        #Sending parts of vectors to the process
        comm.send(data1[startIndex:endIndex],dest = i)
        comm.send(data2[startIndex:endIndex],dest = i)
        #Receiving answer
        tempVec = comm.recv(source = i)
        #Storing results from all processes in a new vector
        resultVec.append(tempVec.tolist())
        stop = MPI.Wtime() 
        print("Time required :",stop-start)
    #print("Vector with sum : ",sum(resultVec,[]))

        
vecSize = 10**5
#Consider worker 0 to initialize the data and broadcast to other workers
def vectorAddition():
    if rank == 0:
        #Initializing vector ar root process
        vec1 = np.random.rand(vecSize)
        vec2 = np.random.rand(vecSize)
        #print("original",vec1,"\n",vec2)
        #Using send and recv to distribute vectors to all processes
        broadcast(vec1,vec2,size)
    
    
#For all workers except 0, receiving the data sent by source
    else :
        dataRecv1 = comm.recv(source = 0)
        dataRecv2 = comm.recv(source = 0)
        print("Vector size at {} is {}".format(rank,len(dataRecv1)))
        #Computing the sum and sending it to root process
        comm.send(dataRecv1+dataRecv2,dest = 0)

#To ensure synchronization
comm.barrier()
#Time with overheads(waiting time)
tic = MPI.Wtime()
vectorAddition()
#End of process, stop timer
comm.barrier()
tac = MPI.Wtime()

print("Total time ", tac-tic)

In [ ]:
!mpiexec -n 3 python Ex1a.py

b) Find an average of numbers in a vector.

In [ ]:
%%file Ex1b.py
#Import numpy,mpi4py and time packages
import numpy as np
from mpi4py import MPI
import time

#Setting alias to MPI.COMM_WORLD for easier reference in the code
comm = MPI.COMM_WORLD

#Computing the number of processes(size) and rank(current process)
size = comm.Get_size()
rank = comm.Get_rank()

print(comm,size,rank)


vecSize = 10**7
result = []

#Function to broadcast vectors to all receiving processes
def broadcastSum(data,size):
    
    for i in range(1,size):
        tic = time.time()
        #Evenly spacing the vector among the available workers
        startIndex = int((i-1)*(len(data)/(size-1)))
        endIndex = int(((len(data)/(size-1))*i))
        ##print("Index",startIndex,endIndex)
        if endIndex > len(data):
            endIndex = int(len(data))
        ##print("Data Sent :",data[startIndex:endIndex])
        #Sending partitioned data to available processes
        comm.send(data[startIndex:endIndex],dest = i)
        temp = comm.recv(source = i)
        result.append(temp)
        tac = time.time()
        print("Time without overhead ",tac-tic)
    print("Average",np.sum(np.array(result))/vecSize)
    

#Capture initial time
comm.barrier()
start = time.time()

#Consider worker 0 to initialize the data and broadcast to other workers
if rank == 0:
    vec = np.random.rand(vecSize)
    #print("original",vec)
    broadcastSum(vec,size)
    
else:
    dataRecv = comm.recv(source = 0)
    comm.send(np.sum(np.array(dataRecv)),dest = 0)
    

#End of process, stop timer   
comm.barrier()
stop = time.time() 

print("Time required :",stop-start)

In [ ]:
!mpiexec -n 4 python Ex1b.py

Exercise 2: Parallel Matrix Vector multiplication using MPI (7 Points)

        In this exercise you have to work with a matrix A ∈ R N ×N and two vectors b ∈ R N , c ∈ R N . Initialize
the matrix A and vector b with random numbers (can be either integers or floating points). The vector
c will store result of A × b.
In case of matrix vector multiplication, you will experiment with three different sizes of matrices i.e.
N = {10 2 , 10 3 , 10 4 }. [note: your matrix will be N × N , which means in case 1 you will have matrix
dimension 100x100]. You will have to run experiments with varying number of workers, i.e. if your system
has P workers than run experiments with workers = {1, 2, . . . P } for each matrix size given above. You
have to time your code and present it in a table.
Implement parallel matrix vector multiplication using MPI point-to-point communication i.e. Send
and Recv. Explain your logic in the report i.e. how the matrix and vectors are divided (distributed)
among workers, what is shared among them, how is the work distributed, what individual worker will
do and what master worker will do.

In [19]:
%%file Ex2.py
#Import numpy,mpi4py and time packages
import numpy as np
from mpi4py import MPI
import time

n = 10**4
product = []
    

#Setting alias to MPI.COMM_WORLD for easier reference in the code
comm = MPI.COMM_WORLD

#Computing the number of processes(size) and rank(current process)
size = comm.Get_size()
rank = comm.Get_rank()

print(comm,size,rank)

#Function to broadcast vectors to all receiving processes
def broadcast(matrix,vector,size):
    for i in range(1,size):
        
        #Evenly spacing the vector among the available workers
        startIndex = int((i-1)*(matrix.shape[0]/(size-1)))
        endIndex = int(((matrix.shape[0]/(size-1))*i))
        ##print("Index",startIndex,endIndex)
        if endIndex > matrix.shape[0]:
            endIndex = int(matrix.shape[0])
        ##print("Data Sent :",data[startIndex:endIndex])
        tic = time.time()
        comm.send(matrix[startIndex:endIndex,],dest = i)
        comm.send(vector,dest = i)
        temp = comm.recv(source = i)
        product.append(temp.tolist())
        print("Time without overhead ",time.time()-tic)
    #print("Result",sum(product,[]))


def process():
#Consider worker 0 to initialize the data and broadcast to other workers
    if rank == 0:
        matrix = np.random.rand(n,n)
        #print("Matrix",matrix)
        vector = np.random.rand(n)
        #print("Vector",vector)
        broadcast(matrix,vector,size)
    
    else:
        matrixRecv = comm.recv(source = 0)
        vectorRecv = comm.recv(source = 0)
        part = np.matmul(matrixRecv,vectorRecv)
        comm.send(part,dest = 0)
    
#Capture initial time
comm.barrier()
start = time.time()
process()
#End of process, stop timer  
comm.barrier()
stop = time.time() 
print("Time required :",stop-start)

Overwriting Ex2.py


In [24]:
!mpiexec -n 4 python Ex2.py

<mpi4py.MPI.Intracomm object at 0x7f5ba43c9ac8> 4 0
Time without overhead  0.6799919605255127
Time without overhead  0.6668262481689453
Time without overhead  0.6557085514068604
Time required : 4.138477563858032
<mpi4py.MPI.Intracomm object at 0x7faa4860bac8> 4 1
Time required : 4.1384875774383545
<mpi4py.MPI.Intracomm object at 0x7fe2c1189ac8> 4 2
Time required : 4.1384899616241455
<mpi4py.MPI.Intracomm object at 0x7fe381da4ac8> 4 3
Time required : 4.138485908508301


Exercise 3: Parallel Matrix Operation using MPI (8 Points)
    
    In this exercise you have to work with three matrices (A ∈ R N ×N , B ∈ R N ×N , C ∈ R N ×N ) i.e each
matrix having size N ×N . Initialize your matrices A and B with random numbers (can be either integers
or floating points). Matrix C will store result of A × B.
In case of matrix multiplication, you will experiment with three different sizes of matrices i.e. N =
{10 2 , 10 3 , 10 4 }. [note: your matrix will be N × N , which means in case 1 you will have matrices with
dimension 100x100]. You will have to run experiments with varying number of workers, i.e. if your system
has P workers than run experiments with workers = {1, 2, . . . P } for each matrix size given above. You
have to time your code and present it in a table.
Implement parallel matrix matrix multiplication using MPI collective communication. Explain your
logic in the report i.e. how the matrices are divided (distributed) among workers, what is shared among
them, how is the work distributed, what individual worker will do and what master worker will do. Per-
form experiments with varying matrix sizes and varying number of workers. You can look at the imple-
mentation provided in the lecture (slide 48) https://www.ismll.uni-hildesheim.de/lehre/bd-17s/
script/bd-01-parallel-computing.pdf

In [41]:
%%file Ex3.py
#Importing the required packages
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

row =10**2
col =10**2
pdt = None

sendbuf = None

tic = MPI.Wtime()
#Prcoesses to be done at root
if rank == 0:
    sendbuf = np.random.rand(row,col)
    matrixB = np.random.rand(row,col)
    
    #Calculating split and displacements for ScatterV and GatherV as scatter and gather cannot deal when
    #number of rows cannot be evenly distributed to workers 
    split = np.array_split(sendbuf,size,axis=0)
    #print("split",type(split))
    splits = []

    for i in range(0,len(split),1):
        splits = np.append(splits, len(split[i]))
        
    splitI = splits*col
    dispI = np.insert(np.cumsum(splitI),0,0)[0:-1]

    splitO = splits*col
    dispO = np.insert(np.cumsum(splitO),0,0)[0:-1]
    
    #print("Mat A",sendbuf)
    #print("Mat B",matrixB)
    matrixA = np.empty((int(row/size),col))
    pdt = np.empty((row,col))
    #print("rank o ",sendbuf)
    
else:
    splitI = None
    dispI = None
    splitO = None
    dispO = None
    split = None
    
    matrixB = np.empty((row,col))

#Bcasting all variables for scatter and gather as they were computed only in root process
split = comm.bcast(split, root=0) 
splitI = comm.bcast(splitI, root = 0)
dispI = comm.bcast(dispI, root = 0)
splitO = comm.bcast(splitO, root = 0)
dispO = comm.bcast(dispO, root = 0)

#print("After bcast",type(split),type(rank),type(col))
matrixA = np.empty((split[rank].shape[0],col))

#Scattering matrix A
comm.Scatterv([sendbuf,splitI, dispI,MPI.DOUBLE], matrixA, root=0)
#Bcasting matrix B
matrixB = comm.bcast(matrixB, root = 0)


comm.barrier()
#Performing matrix multiplication at all processes including root
temp = np.matmul(matrixA,matrixB)
#Gathering the results at root process
comm.Gatherv(temp,[pdt,splitO,dispO,MPI.DOUBLE], root=0)

comm.barrier()
toc = MPI.Wtime()

#print("rank",rank)
print("Time at Rank",rank,"Time",toc-tic)
#print("Scatter",matrixA)
#print("Bcast",matrixB)
#print("Pdt",pdt)

Overwriting Ex3.py


In [42]:
!mpiexec -n 3 python Ex3.py

Time at Rank 0 Time 0.022871017456054688
Time at Rank 1 Time 0.01910400390625
Time at Rank 2 Time 0.05176091194152832


In [ ]:
?MPI.Scatterv

In [56]:
from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

n = 10**3

tic = time.time()
if rank == 0:
    vmatrix = np.random.rand(n,n)
    #print("Matrix",matrix)
    vector = np.random.rand(n,n)
    new = np.matmul(vmatrix,vector)
print("Time",time.time()-tic)

Time 0.20702242851257324
